In [ ]:
import numpy as np                  # Version: 1.24.3
import pandas as pd                 # Version: 1.5.3
from   pandas.api.types import CategoricalDtype
import geopandas as gpd             # Version: 1.4.0
import fiona                        # Version: 1.16.0
import scipy                        # Version: 1.10.1
from   scipy.stats import zscore, genpareto

# Python version: 3.11.4
import os
import calendar

In [3]:
notebook_dir = os.getcwd()
parent_dir   = os.path.dirname(notebook_dir)
target_dir   = os.path.join(parent_dir, "Data")

state_order = [
    "Australian Capital Territory", "New South Wales", 
    "Queensland", "South Australia", "Tasmania", "Victoria", "Western Australia"
]

### **DATASET 1**: Historical Wildfires

`Bushfire_Boundaries_Historical_2024_V3.gdb`

**Australian Government** (**Geoscience Australia**) [Bushfire Historical Extents - Version 3.0](https://dx.doi.org/10.26186/149978)
- Temporal extent: 30-12-1899 to 29-08-2024
- Does not include information about the Northern Territory
- Australian Capital Territory is very small compared to the remaining represented regions and therefore (among other reasons) has less instances

In [4]:
gdb_path = os.path.join(target_dir, "Bushfire_Boundaries_Historical_2024_V3.gdb")
layers   = fiona.listlayers(gdb_path)
print("Available Layers:", layers, "\n")

# # (optional) Uncomment to load from dataset instead of parquet file
# # However, parquet is about 10x faster
# get_layer = layers[0]
# gdf = gpd.read_file(gdb_path, layer = get_layer)
# gdf.to_parquet(os.path.join(target_dir, "bushfire_boundaries.parquet"))

# Load from parquet file
gdf = gpd.read_parquet(os.path.join(target_dir, "bushfire_boundaries.parquet"))

print(gdf.columns)
gdf.drop(["capture_date", "extinguish_date", "capt_method", "perim_km", "agency", "Shape_Length", "Shape_Area", "geometry"], axis = 1, inplace = True)

Available Layers: ['Bushfire_Boundaries_Historical_V3'] 

Index(['fire_id', 'fire_name', 'ignition_date', 'capture_date',
       'extinguish_date', 'fire_type', 'ignition_cause', 'capt_method',
       'area_ha', 'perim_km', 'state', 'agency', 'Shape_Length', 'Shape_Area',
       'geometry'],
      dtype='object')


#### **Ignition Date**

In [ ]:
### FILTER IGNITION DATE ###

gdf["ignition_date"] = pd.to_datetime(gdf["ignition_date"], errors = "coerce")
gdf_filtered         = gdf[(gdf["ignition_date"].dt.year >= 2005) & (gdf["ignition_date"].dt.year <= 2020)]
print("Number of instances:", len(gdf_filtered), "\n")
# Check missing values
missing_counts = gdf_filtered.isnull().sum()
print(missing_counts, "\n")
# Distribution across states
value_counts = gdf_filtered["state"].value_counts()
print(value_counts)

Number of instances: 129428 

fire_id            16496
fire_name          46076
ignition_date          0
fire_type              0
ignition_cause    107508
area_ha                0
state                  0
dtype: int64 

state
WA (Western Australia)                54466
VIC (Victoria)                        47368
NSW (New South Wales)                  9985
QLD (Queensland)                       8837
TAS (Tasmania)                         5829
SA (South Australia)                   2363
ACT (Australian Capital Territory)      580
Name: count, dtype: int64


#### **Fire Type**
- **Bushfire**: Unplanned vegetation fire. A generic term which includes grass fires, forest fires and scrub fires both with and without a suppression objective. Also known as wildfire, accident, arson, lightning.
- **Prescribed Burn**: The controlled application of fire under specified environmental conditions to a predetermined area and at the time, intensity, and rate of spread required to attain planned resource management objectives. Also known as planned burning, fuel reduction, traditional owner, ecological, hazard reduction
- **Unknown**: Fire type is undetermined. ([Source](https://www.arcgis.com/home/item.html?id=912492b1683e494e8efbe84573ce6d70))

In [ ]:
### FILTER FIRE TYPE ###

value_counts = gdf_filtered["fire_type"].value_counts()
print(value_counts, "\n")

gdf_unknown = gdf_filtered[gdf_filtered["fire_type"] == "Unknown"]
gdf_unknown = gdf_unknown.copy()
# Distribution across states (fire_type == Unknown)
value_counts = gdf_unknown["state"].value_counts()
print(value_counts, "\n")

gdf_filtered = gdf_filtered[gdf_filtered["fire_type"] == "Bushfire"]
print("Number of instances:", len(gdf_filtered), "\n")
# Check missing values
missing_counts = gdf_filtered.isnull().sum()
print(missing_counts, "\n")
# Distribution across states (fire_type == Bushfire)
value_counts = gdf_filtered["state"].value_counts()
print(value_counts)

fire_type
Bushfire    31898
Name: count, dtype: int64 

Series([], Name: count, dtype: int64) 

Number of instances: 31898 

fire_id            4112
fire_name          2390
ignition_date         0
fire_type             0
ignition_cause    10115
area_ha               0
state                 0
dtype: int64 

state
NSW (New South Wales)                 9942
WA (Western Australia)                8709
VIC (Victoria)                        4899
QLD (Queensland)                      3928
TAS (Tasmania)                        3053
SA (South Australia)                  1288
ACT (Australian Capital Territory)      79
Name: count, dtype: int64


#### **Ignition Cause**

- **Accidental**: Fires that are not the result of a deliberate (intentional) act.
- **Undetermined**: Fires that have not yet been investigated, under investigation or fires that have been investigated and the cause is not proven to an acceptable level of certainty.
- **Incendiary**: Fires result from deliberate acts, intentional actions, or circumstances for the fire to occur in areas where it should not have occurred.
- **Natural**: Fires that ignite without human intervention. ([Source](https://www.arcgis.com/home/item.html?id=912492b1683e494e8efbe84573ce6d70))

In [12]:
### FILTER IGNITION CAUSE ###

value_counts = gdf_filtered["ignition_cause"].value_counts()
print(value_counts, "\n")

gdf_undetermined = gdf_filtered[gdf_filtered["ignition_cause"] == "Undetermined"]
# Distribution across states (ignition_cause == Undetermined)
value_counts = gdf_undetermined["state"].value_counts()
print(value_counts, "\n")

# Distribution across states
value_counts = gdf_filtered["state"].value_counts()
print(value_counts, "\n")
# Distribution across states (ignition_cause == NA)
gdf_NA_cause = gdf_filtered[gdf_filtered["ignition_cause"].isna()]
value_counts = gdf_NA_cause["state"].value_counts()
print(value_counts)

ignition_cause
Accidental      5637
Incendiary      5572
Undetermined    5523
Natural         5051
Name: count, dtype: int64 

state
NSW (New South Wales)                 3610
WA (Western Australia)                1110
TAS (Tasmania)                         798
ACT (Australian Capital Territory)       5
Name: count, dtype: int64 

state
NSW (New South Wales)                 9942
WA (Western Australia)                8709
VIC (Victoria)                        4899
QLD (Queensland)                      3928
TAS (Tasmania)                        3053
SA (South Australia)                  1288
ACT (Australian Capital Territory)      79
Name: count, dtype: int64 

state
VIC (Victoria)          4899
QLD (Queensland)        3928
SA (South Australia)    1288
Name: count, dtype: int64


Victoria, Queensland, and South Australia did not record ignition cause. ([Source](https://zenodo.org/records/8280702))

Going forward, the data is therefore split into two sets, one has all seven states, the other has only the four states for which the ignition cause was recorded.

In [14]:
gdf_imputed = gdf_filtered
gdf_imputed.fillna({"ignition_cause": "Not recorded"}, inplace = True)

### SET 2 ###
gdf_7states_filtered = gdf_imputed[gdf_imputed["ignition_cause"].isin(["Natural", "Not recorded"])]
print("Number of instances:", len(gdf_7states_filtered), "\n")
# Check missing values
missing_counts = gdf_7states_filtered.isnull().sum()
print(missing_counts, "\n")
# Distribution across states
value_counts = gdf_7states_filtered["state"].value_counts()
print(value_counts, "\n")

### SET 1 ###
gdf_4states_filtered = gdf_filtered[gdf_filtered["ignition_cause"] == "Natural"]
print("Number of instances:", len(gdf_4states_filtered), "\n")
# Check missing values
missing_counts = gdf_4states_filtered.isnull().sum()
print(missing_counts, "\n")
# Distribution across states
value_counts = gdf_4states_filtered["state"].value_counts()
print(value_counts)

Number of instances: 15166 

fire_id           4065
fire_name         1326
ignition_date        0
fire_type            0
ignition_cause       0
area_ha              0
state                0
dtype: int64 

state
VIC (Victoria)                        4899
QLD (Queensland)                      3928
NSW (New South Wales)                 2672
WA (Western Australia)                2132
SA (South Australia)                  1288
TAS (Tasmania)                         229
ACT (Australian Capital Territory)      18
Name: count, dtype: int64 

Number of instances: 5051 

fire_id           15
fire_name         83
ignition_date      0
fire_type          0
ignition_cause     0
area_ha            0
state              0
dtype: int64 

state
NSW (New South Wales)                 2672
WA (Western Australia)                2132
TAS (Tasmania)                         229
ACT (Australian Capital Territory)      18
Name: count, dtype: int64


#### **Partial Entries**

In [16]:
### COMPILE PARTIAL ENTRIES ###

gdf_7states_filtered = gdf_7states_filtered.copy() 
gdf_7states_filtered.fillna({"fire_name": "N.A."}, inplace = True)
gdf_7states_filtered.fillna({"fire_id": "N.A."}, inplace = True)

# Based on same ignition date, state, fire name, and fire ID
duplicates = gdf_7states_filtered.assign(
    duplicates_check = gdf_7states_filtered["ignition_date"].astype(str) + "_" +
                       gdf_7states_filtered["state"] + "_" +
                       gdf_7states_filtered["fire_name"].astype(str) +
                       gdf_7states_filtered["fire_id"].astype(str)
).duplicated(subset = ["duplicates_check"], keep = False)

print("Potential partial entries:", duplicates.sum())
display(gdf_7states_filtered.loc[duplicates, ["ignition_date", "state", "fire_name", "fire_id", "area_ha"]].head(10))

Potential partial entries: 3482


,ignition_date,state,fire_name,fire_id,area_ha
26069,2009-10-23,NSW (New South Wales),Pera Valley,27874,3.0
26076,2009-10-23,NSW (New South Wales),Pera Valley,27874,222.0
26086,2009-10-24,NSW (New South Wales),Five Snake Creek,27932,9.0
26094,2009-10-24,NSW (New South Wales),Five Snake Creek,27932,4.0
26096,2009-10-24,NSW (New South Wales),New England Gully Rd,27912,3.0
26097,2009-10-24,NSW (New South Wales),New England Gully Rd,27912,9.0
26103,2009-11-01,NSW (New South Wales),Moonan Brook Fire,28090,521.0
26104,2009-11-01,NSW (New South Wales),Moonan Brook Fire,28090,779.0
26105,2009-11-01,NSW (New South Wales),Moonan Brook Fire,28090,688.0
26106,2009-11-01,NSW (New South Wales),Moonan Brook Fire,28090,29.0


In [17]:
compiled_7states = gdf_7states_filtered.groupby(["ignition_date", "state", "fire_name", "fire_id"], as_index = False).agg({
    "area_ha": "sum",
    "fire_type": "first",
    "ignition_cause": "first"
})

print("Number of instances:", len(compiled_7states))
display(compiled_7states.tail(10))

# Check missing values
missing_counts = compiled_7states.isnull().sum()
print(missing_counts, "\n")
# Distribution across states
value_counts = compiled_7states["state"].value_counts()
print(value_counts)

Number of instances: 12134


,ignition_date,state,fire_name,fire_id,area_ha,fire_type,ignition_cause
12124,2020-12-31,NSW (New South Wales),"Judds_Carmichels Rd, Savernake",403888,229.0,Bushfire,Natural
12125,2020-12-31,NSW (New South Wales),"The Escort Way, Boree",403852,40.0,Bushfire,Natural
12126,2020-12-31,NSW (New South Wales),"Thelangerin Rd, Hay",403865,104.0,Bushfire,Natural
12127,2020-12-31,SA (South Australia),Laurie Park,0,11.0,Bushfire,Not recorded
12128,2020-12-31,SA (South Australia),Lower Light,0,1.0,Bushfire,Not recorded
12129,2020-12-31,SA (South Australia),Ulyebury,0,21.0,Bushfire,Not recorded
12130,2020-12-31,WA (Western Australia),Delangrafft,BF 2020 DON 010,48.0,Bushfire,Natural
12131,2020-12-31,WA (Western Australia),Jilbadjie,BF 2020 WHB 007,315.0,Bushfire,Natural
12132,2020-12-31,WA (Western Australia),Laurina Road,BF 2020 ALB 004,587.0,Bushfire,Natural
12133,2020-12-31,WA (Western Australia),Maranup Ford,BF 2020 BWD 023,69.0,Bushfire,Natural


ignition_date     0
state             0
fire_name         0
fire_id           0
area_ha           0
fire_type         0
ignition_cause    0
dtype: int64 

state
QLD (Queensland)                      3922
NSW (New South Wales)                 2521
VIC (Victoria)                        2336
WA (Western Australia)                2122
SA (South Australia)                   986
TAS (Tasmania)                         229
ACT (Australian Capital Territory)      18
Name: count, dtype: int64


#### **Burned Area (area ha)**

In [18]:
print("Number of instances (total):", len(compiled_7states["area_ha"]), "\n")
value_counts = compiled_7states["state"].value_counts() 
print(value_counts, "\n")

temp = compiled_7states[compiled_7states["area_ha"] == 0]
perc = (len(temp) / len(compiled_7states["area_ha"])) * 100
print("Number of instances (area_ha == 0):", len(temp), f"(~{round(perc, 0)}%)", "\n")
value_counts = temp["state"].value_counts() 
print(value_counts)

Number of instances (total): 12134 

state
QLD (Queensland)                      3922
NSW (New South Wales)                 2521
VIC (Victoria)                        2336
WA (Western Australia)                2122
SA (South Australia)                   986
TAS (Tasmania)                         229
ACT (Australian Capital Territory)      18
Name: count, dtype: int64 

Number of instances (area_ha == 0): 1235 (~10.0%) 

state
WA (Western Australia)                364
VIC (Victoria)                        290
NSW (New South Wales)                 208
QLD (Queensland)                      207
SA (South Australia)                   83
TAS (Tasmania)                         77
ACT (Australian Capital Territory)      6
Name: count, dtype: int64


Instances where the burned area (area_ha) is recorded as 0 can arise from various scenarios:
- These entries might represent events where a fire was reported or detected, but no significant area was burned. This could occur due to successful suppression efforts.
- Some datasets have a minimum mapping unit, meaning fires smaller than a certain threshold aren't recorded with specific area values. For example, in certain studies, a zero value indicates that an area smaller than 1 hectare was burned. https://core.ac.uk/download/pdf/55609027.pdf

Because these instances show no specific patterns across different regions, it is not likely that the 0-values are due to non-recording. Therefore the assumption is made that they are part of the distribution and retained in the outlier analysis.

#### **Outlier Analysis**

In [27]:
### DAILY AGGREGATION ###
# Aggregate to day + state level
aggregate = compiled_7states.groupby(["ignition_date", "state"], sort=False).agg(
    total_area_ha=("area_ha", "sum"),
    fire_count=("area_ha", "count"),
    ignition_cause=("ignition_cause", "first")
).reset_index()

### OUTLIER ANALYSIS ###
# Perform log transformation to handle skewed burned area distribution
# And compute state-wise Z-scores for log-transformed burned area
aggregate["log_total_area_ha"] = np.log1p(aggregate["total_area_ha"])
aggregate["log_zscore"] = aggregate.groupby("state")["log_total_area_ha"].transform(zscore)

# Generalized Pareto Distribution (GPD) for EVT-based outlier detection
# Fits a GPD to the excesses over a high threshold
def fit_evt_threshold(data, threshold_percentile=95):
    threshold = np.percentile(data, threshold_percentile)
    excesses = data[data > threshold] - threshold
    if len(excesses) > 0:
        shape, _, scale = genpareto.fit(excesses)
        return shape, scale, threshold
    return None, None, threshold
# Apply EVT thresholding per state
evt_results = aggregate.groupby("state")["total_area_ha"].apply(lambda x: fit_evt_threshold(x))
# Extract EVT threshold results into the dataframe
aggregate["evt_shape"] = aggregate["state"].map(lambda s: evt_results[s][0])
aggregate["evt_scale"] = aggregate["state"].map(lambda s: evt_results[s][1])
aggregate["evt_threshold"] = aggregate["state"].map(lambda s: evt_results[s][2])
# Identify EVT outliers: if burned_area > (threshold + 3 * scale)
aggregate["evt_outlier"] = aggregate.apply(
    lambda row: row["total_area_ha"] > (row["evt_threshold"] + 3 * row["evt_scale"]) if row["evt_scale"] is not None else False, axis=1
)

# Outlier type
def categorize_outlier(row):
    if row["evt_outlier"] and abs(row["log_zscore"]) > 3:
        return "Both"
    elif row["evt_outlier"]:
        return "EVT outlier"
    else:
        return "None"
aggregate["outlier_type"] = aggregate.apply(categorize_outlier, axis=1)

# Summary output
outliers_evt = aggregate[aggregate['outlier_type'] == "EVT outlier"]
outliers_zscore = aggregate[aggregate['outlier_type'] == "Both"]

print("Total day/state instances:", len(aggregate), "\n")
print("EVT outliers:               ", len(outliers_evt))
print("Log Z-score outliers:       ", len(outliers_zscore))
print("Total outliers:             ", len(outliers_evt) + len(outliers_zscore))

# DF aggregate
aggregate = aggregate[["ignition_date", "state", "total_area_ha", "log_total_area_ha", "fire_count", "outlier_type", "ignition_cause"]]
display(aggregate.head())

Total day/state instances: 5478 

EVT outliers:                49
Log Z-score outliers:        13
Total outliers:              62


,ignition_date,state,total_area_ha,log_total_area_ha,fire_count,outlier_type,ignition_cause
0,2005-01-01,QLD (Queensland),2379.0,7.774856,1,None,Not recorded
1,2005-01-01,VIC (Victoria),12722.0,9.451167,26,None,Not recorded
2,2005-01-01,WA (Western Australia),219.0,5.393628,1,None,Natural
3,2005-01-02,SA (South Australia),4.0,1.609438,1,None,Not recorded
4,2005-01-02,WA (Western Australia),30885.0,10.338058,6,None,Natural


In [31]:
# Dictionary to replace abbreviations with full names
state_order = ["New South Wales", "Queensland", "South Australia", "Tasmania", "Victoria", "Western Australia"]
state_name_mapping = {
    "ACT (Australian Capital Territory)": "Australian Capital Territory",
    "NSW (New South Wales)": "New South Wales",
    "QLD (Queensland)": "Queensland",
    "SA (South Australia)": "South Australia",
    "TAS (Tasmania)": "Tasmania",
    "VIC (Victoria)": "Victoria",
    "WA (Western Australia)": "Western Australia"
}
aggregate["state"] = aggregate["state"].replace(state_name_mapping)
aggregate["state"] = aggregate["state"].astype(CategoricalDtype(categories=state_order, ordered=True))


state_order_star = ["New South Wales", "Queensland*", "South Australia*", "Tasmania", "Victoria*", "Western Australia"]
state_order_map = dict(zip(state_order, state_order_star))

# Prepare data
outliers = aggregate[aggregate["outlier_type"] != "None"].copy()
outliers["ignition_date"] = pd.to_datetime(outliers["ignition_date"])
outliers["state"] = pd.Categorical(outliers["state"], categories=state_order, ordered=True)
outliers["outlier_type"] = outliers["outlier_type"].replace({"Both": "Z-score outlier"})

# Add split label for summary
outliers["year"] = outliers["ignition_date"].dt.year
outliers["split"] = pd.cut(
    outliers["year"],
    bins=[2004, 2016, 2018, 2020],
    labels=["Train", "Validation", "Test"]
)
summary = outliers.groupby(["split", "state"], observed=False).size().unstack(fill_value=0).reindex(["Train", "Validation", "Test"])
summary = summary.rename(columns=state_order_map)  # Pretty column names
display(summary)

state,New South Wales,Queensland*,South Australia*,Tasmania,Victoria*,Western Australia
split,,,,,,
Train,1,14,7,0,12,4
Validation,0,3,0,0,0,0
Test,8,4,0,1,6,2


In [37]:
aggregate["year"] = aggregate["ignition_date"].dt.year
print("Total S2:  ", len(aggregate))
train = aggregate[aggregate["year"] < 2017]
valid = aggregate[(aggregate["year"] >= 2017) & (aggregate["year"] < 2019)]
test  = aggregate[aggregate["year"] >= 2019]
print("Train:     ", len(train[~(train["outlier_type"] == "None")]), "(", round(len(train) / len(aggregate) * 100, 1), "% )")
print("Validation:", len(valid[~(valid["outlier_type"] == "None")]), " (", round(len(valid) / len(aggregate) * 100, 1), "% )")
print("Test:      ", len(test[~(test["outlier_type"] == "None")]), "(", round(len(test) / len(aggregate) * 100, 1), "% )\n")

aggregate4 = aggregate[aggregate["state"].isin(["Australian Capital Territory", "New South Wales", "Tasmania", "Western Australia"])]
print("Total S1:  ", len(aggregate4))
train = aggregate4[aggregate4["year"] < 2017]
valid = aggregate4[(aggregate4["year"] >= 2017) & (aggregate4["year"] < 2019)]
test  = aggregate4[aggregate4["year"] >= 2019]
print("Train:     ", len(train[~(train["outlier_type"] == "None")]), " (", round(len(train) / len(aggregate4) * 100, 1), "% )")
print("Validation:", len(valid[~(valid["outlier_type"] == "None")]), " (", round(len(valid) / len(aggregate4) * 100, 1), "% )")
print("Test:      ", len(test[~(test["outlier_type"] == "None")]), "(", round(len(test) / len(aggregate4) * 100, 1), "% )")

Total S2:   5478
Train:      38 ( 67.9 % )
Validation: 3  ( 16.1 % )
Test:       21 ( 16.0 % )

Total S1:   1853
Train:      5  ( 64.2 % )
Validation: 0  ( 17.3 % )
Test:       11 ( 18.5 % )


#### **Spatial**

Below are the total land areas of the states and territories, including the percentage contribution to the total land mass. ([Source](https://www.ga.gov.au/scientific-topics/national-location-information/dimensions/area-of-australia-states-and-territories))

In [38]:
data = [
    {"State/Territory": "WA (Western Australia)", "%": 32.9, "Total_area_km2": 2527013},
    {"State/Territory": "QLD (Queensland)", "%": 22.5, "Total_area_km2": 1729742},
    {"State/Territory": "NT (Northern Territory)", "%": 17.5, "Total_area_km2": 1347791},
    {"State/Territory": "SA (South Australia)", "%": 12.8, "Total_area_km2": 984321},
    {"State/Territory": "NSW (New South Wales)", "%": 10.4, "Total_area_km2": 801150},
    {"State/Territory": "VIC (Victoria)", "%": 3.0, "Total_area_km2": 227444},
    {"State/Territory": "TAS (Tasmania)", "%": 0.9, "Total_area_km2": 68401},
    {"State/Territory": "ACT (Australian Capital Territory)", "%": 0.1, "Total_area_km2": 2358}
]

land_areas = pd.DataFrame(data)
land_areas["Total_area_ha"] = land_areas["Total_area_km2"] * 100
display(land_areas)

,State/Territory,%,Total_area_km2,Total_area_ha
0,WA (Western Australia),32.9,2527013,252701300
1,QLD (Queensland),22.5,1729742,172974200
2,NT (Northern Territory),17.5,1347791,134779100
3,SA (South Australia),12.8,984321,98432100
4,NSW (New South Wales),10.4,801150,80115000
5,VIC (Victoria),3.0,227444,22744400
6,TAS (Tasmania),0.9,68401,6840100
7,ACT (Australian Capital Territory),0.1,2358,235800


#### **Fire to No-Fire Distribution**

In [ ]:
years         = range(2005, 2021)
days_per_year = {year: 366 if calendar.isleap(year) else 365 for year in years}
total_days    = sum(days_per_year.values())

total_7states = aggregate.groupby(["state"], observed=False).agg(
    total_firedays     = ("ignition_date", "count"),
    ignition_cause = ("ignition_cause", "first")
).reset_index()
total_7states["(%)_fire"]    = ((total_7states["total_firedays"] / total_days) * 100).round(1)
total_7states["(%)_no-fire"] = 100 - total_7states["(%)_fire"]
display(total_7states)

value_counts = aggregate["state"].value_counts()
print(value_counts, "\n")

days_4regions = total_days * 4
days_7regions = total_days * 7
print(days_per_year)
print("Total days 2005-2020:", total_days)
print("Set 1 instances:     ", days_4regions)
print("Set 2 instances:     ", days_7regions)
regions4 = 888+852+113+16
p4       = (regions4/days_4regions) * 100
regions7 = regions4+1951+589+1069
p7       = (regions7/days_7regions) * 100
print("Set 1 fire:          ", regions4, "( ", round(p4,1), "% )")
print("Set 2 fire:          ", regions7, "(", round(p7,1), "% )")

,state,total_firedays,ignition_cause,(%)_fire,(%)_no-fire
0,New South Wales,852,Natural,14.6,85.4
1,Queensland,1951,Not recorded,33.4,66.6
2,South Australia,589,Not recorded,10.1,89.9
3,Tasmania,113,Natural,1.9,98.1
4,Victoria,1069,Not recorded,18.3,81.7
5,Western Australia,888,Natural,15.2,84.8


state
Queensland           1951
Victoria             1069
Western Australia     888
New South Wales       852
South Australia       589
Tasmania              113
Name: count, dtype: int64 

{2005: 365, 2006: 365, 2007: 365, 2008: 366, 2009: 365, 2010: 365, 2011: 365, 2012: 366, 2013: 365, 2014: 365, 2015: 365, 2016: 366, 2017: 365, 2018: 365, 2019: 365, 2020: 366}
Total days 2005-2020: 5844
Set 1 instances:      23376
Set 2 instances:      40908
Set 1 fire:           1869 (  8.0 % )
Set 2 fire:           5478 ( 13.4 % )


**Important to note**: `(%)_fire` is the percentage of days across 2005-2020 where a wildfire ignited, conversely, `(%)_no-fire` is the percentage of days no wildfire ignited. 

#### **Save Data**

In [ ]:
### SAVE DATA ###

exp_csv_path = os.path.join(target_dir, "01_historical_wildfires.csv")
aggregate.to_csv(exp_csv_path, index = True)
display(aggregate.head(5))

,ignition_date,state,total_area_ha,log_total_area_ha,fire_count,outlier_type,ignition_cause
0,2005-01-01,Queensland,2379.0,7.774856,1,None,Not recorded
1,2005-01-01,Victoria,12722.0,9.451167,26,None,Not recorded
2,2005-01-01,Western Australia,219.0,5.393628,1,None,Natural
3,2005-01-02,South Australia,4.0,1.609438,1,None,Not recorded
4,2005-01-02,Western Australia,30885.0,10.338058,6,None,Natural


### **DATASET 2**: Historical Weather Metrics

Output of `ERA5_script` 
- 192 individual `.nc` files: with each file corresponding to a single month between 2005 and 2020 and containing all variables
- Stored in csv format: `era5_stats_all.csv`

In [46]:
csv_path = os.path.join(target_dir, "ERA5", "CSV", "era5_stats_all.csv")
era      = pd.read_csv(csv_path)

csv_path = os.path.join(target_dir, "01_historical_wildfires.csv")
fires    = pd.read_csv(csv_path)

#### **Outlier Analysis**

In [ ]:
### OUTLIER ANALYSIS ###

era["Date"]            = pd.to_datetime(era["Date"])
fires["ignition_date"] = pd.to_datetime(fires["ignition_date"])

target_columns = [col for col in era.columns if any(metric in col for metric in ["Temperature", "Precipitation", "Relative humidity", "Wind speed", "Solar radiation"])]
outliers_all   = []
for state, group in era.groupby("State"):
    group_outliers = group.copy()
    for col in target_columns:
        # Compute Z-scores for each column within the state group
        group_outliers[col + "_zscore"]     = zscore(group[col])
        group_outliers[col + "_is_outlier"] = group_outliers[col + "_zscore"].abs() > 3
    # Keep rows with any outlier in any column
    mask = group_outliers[[c for c in group_outliers.columns if "_is_outlier" in c]].any(axis=1)
    outliers_all.append(group_outliers[mask])

# Combine all outlier rows
outliers_df = pd.concat(outliers_all)
# Merge with wildfire ignition dates (if same state and date)
merged = outliers_df.merge(fires, how="left", left_on=["State", "Date"], right_on=["state", "ignition_date"])
merged["is_fire_day"] = ~merged["ignition_date"].isna()
print("Number of outliers:", len(outliers_df))
print("Percentage:        ", round(len(outliers_df) / len(era) * 100, 1), "%\n")
print(merged["is_fire_day"].value_counts(), "\n")
print("Percentage of fire days that is an outlier:", round(413 / len(fires) * 100, 1), "%")


Number of outliers: 2856
Percentage:         7.0 %

is_fire_day
False    2443
True      413
Name: count, dtype: int64 

Percentage of fire days that is an outlier: 7.5 %


In [ ]:
# Create fire/nonfire outlier subsets
fire_outliers    = merged[merged["is_fire_day"]].copy()
nonfire_outliers = merged[~merged["is_fire_day"]].copy()
# Combine and prepare for event window detection
combined = pd.concat([
    fire_outliers.assign(fire_day=True),
    nonfire_outliers.assign(fire_day=False)
]).sort_values(["State", "Date"]).reset_index(drop=True)
# Assign event_window_id using a stable row-wise approach
event_windows     = []
current_window_id = 0

# Group per state and walk through sorted dates
for state, group in combined.groupby("State"):
    group = group.sort_values("Date").copy()
    group["event_window_id"] = np.nan
    prev_date = None
    for i, row in group.iterrows():
        if prev_date is None or (row["Date"] - prev_date).days > 1:
            current_window_id += 1
        group.at[i, "event_window_id"] = current_window_id
        prev_date = row["Date"]
    event_windows.append(group)

# Recombine all groups with proper window IDs
combined = pd.concat(event_windows).reset_index(drop=True)
# Flag windows that include at least one fire day
window_info = combined.groupby("event_window_id")["fire_day"].any().reset_index()
window_info = window_info.rename(columns={"fire_day": "window_contains_fire"})
combined = combined.merge(window_info, on="event_window_id", how="left")
# Merge based on 'State' and 'Date'
nonfire_outliers = nonfire_outliers.merge(
    combined[["State", "Date", "window_contains_fire"]],
    on=["State", "Date"],
    how="left"
)
nonfire_outliers["is_contiguous_to_fire"] = nonfire_outliers["window_contains_fire"].fillna(False)
print(f"Non-fire outliers in contiguous fire windows: {nonfire_outliers['is_contiguous_to_fire'].sum()} / {len(nonfire_outliers)}")


Non-fire outliers in contiguous fire windows: 179 / 2443


In [ ]:
target_columns = [col for col in era.columns if any(metric in col for metric in [
    "Temperature", "Precipitation", "Relative humidity", "Wind speed", "Solar radiation"])]
# Ensure NaNs are handled as False for filtering
nonfire_outliers["is_contiguous_to_fire"] = nonfire_outliers["is_contiguous_to_fire"].fillna(False)
fire_day_outliers = fire_outliers.copy()
contiguous_outliers = nonfire_outliers[nonfire_outliers["is_contiguous_to_fire"]].copy()
isolated_outliers = nonfire_outliers[~nonfire_outliers["is_contiguous_to_fire"]].copy()

# Label clustered isolated using rolling window
def label_clustered_rolling(df, window_size=5, min_outliers_in_window=2):
    df = df.sort_values(["State", "Date"]).copy()
    df["is_clustered_isolated"] = False
    for state, group in df.groupby("State"):
        group = group.sort_values("Date").reset_index()
        dates = group["Date"]
        is_clustered = np.zeros(len(dates), dtype=bool)
        for i in range(len(dates)):
            start = dates[i]
            end = start + pd.Timedelta(days=window_size - 1)
            mask = (dates >= start) & (dates <= end)
            if mask.sum() >= min_outliers_in_window:
                is_clustered[mask] = True
        df.loc[group["index"], "is_clustered_isolated"] = is_clustered
    return df

# Apply to isolated outliers
isolated_outliers  = label_clustered_rolling(isolated_outliers, window_size=5, min_outliers_in_window=2)
clustered_isolated = isolated_outliers[isolated_outliers["is_clustered_isolated"]].copy()
true_isolated      = isolated_outliers[~isolated_outliers["is_clustered_isolated"]].copy()
# Function to count outliers per variable
def count_outliers_by_variable(df, label=""):
    cols = [col for col in df.columns if col.endswith("_is_outlier")]
    counts = df[cols].sum().sort_values(ascending=False)
    counts.index = [c.replace("_is_outlier", "") for c in counts.index]
    counts.name = label
    return counts
# Count per group
fire_counts       = count_outliers_by_variable(fire_day_outliers, "Fire Days")
contiguous_counts = count_outliers_by_variable(contiguous_outliers, "Contiguous Fire Window")
clustered_counts  = count_outliers_by_variable(clustered_isolated, "Clustered Isolated")
true_isolated_counts = count_outliers_by_variable(true_isolated, "True Isolated")
outlier_summary = pd.concat([fire_counts, contiguous_counts, clustered_counts, true_isolated_counts], axis=1).fillna(0).astype(int)
outlier_summary = outlier_summary.loc[outlier_summary.sum(axis=1).sort_values(ascending=False).index] # Sort by total
print(outlier_summary)
print("\nTotal counts:")
print(outlier_summary.sum())
print(len(clustered_isolated))
print(len(true_isolated))

                               Fire Days  Contiguous Fire Window  \
Precipitation [mm/day]_std            93                      89   
Precipitation [mm/day]_max            97                      86   
Precipitation [mm/day]_sum            80                      94   
Precipitation [mm/day]_min            55                      75   
Wind speed [m/s]_std                  78                       9   
Wind speed [m/s]_min                  27                      24   
Relative humidity [%]_max             77                      17   
Wind speed [m/s]_mean                 37                      21   
Wind speed [m/s]_max                  42                      15   
Temperature [°C]_std                  62                      11   
Relative humidity [%]_std             32                       4   
Relative humidity [%]_min              4                      25   
Relative humidity [%]_mean            14                       5   
Temperature [°C]_max                  15        

In [ ]:
### TREAT OUTLIERS ###

# Set treatment variables
treatment_variables = [col for col in era.columns if any(v in col for v in ['Precipitation', 'Wind speed'])]
era_treated = era.copy()
# Filter training period for true isolated outliers
training_mask = (true_isolated["Date"].dt.year >= 2005) & (true_isolated["Date"].dt.year <= 2016)
true_isolated_train = true_isolated[training_mask].copy()
# Parameters for Winsorization
lower_limit = 0.05
upper_limit = 0.95
change_log  = []
# Treat each variable individually
for var in treatment_variables:
    outlier_flag_col = var + "_is_outlier"
    if outlier_flag_col not in true_isolated_train.columns:
        continue
    for state, group in true_isolated_train.groupby("State"):
        outlier_rows = group[group[outlier_flag_col]]
        if outlier_rows.empty or outlier_rows[var].notna().sum() < 10:
            continue
        # Compute clipping bounds
        lower_bound = outlier_rows[var].quantile(lower_limit)
        upper_bound = outlier_rows[var].quantile(upper_limit)
        for idx, row in outlier_rows.iterrows():
            original_value = row[var]
            clipped_value = np.clip(original_value, lower_bound, upper_bound)
            # Update only the matching row/column in era_treated
            mask = (era_treated["Date"] == row["Date"]) & (era_treated["State"] == row["State"])
            era_treated.loc[mask, var] = clipped_value
            if original_value != clipped_value:
                change_log.append({
                    "Date": row["Date"],
                    "State": row["State"],
                    "Variable": var,
                    "Original": original_value,
                    "Clipped": clipped_value
                })
change_log_df = pd.DataFrame(change_log)

exp_csv_path = os.path.join(target_dir, "02b_change_log.csv")
change_log_df.to_csv(exp_csv_path, index = True)
display(change_log_df.head(5))

,Date,State,Variable,Original,Clipped
0,2006-04-05,Australian Capital Territory,Wind speed [m/s]_mean,6.028696,6.071348
1,2012-09-07,Australian Capital Territory,Wind speed [m/s]_mean,6.992190,6.849015
2,2013-10-10,New South Wales,Wind speed [m/s]_mean,6.046298,6.077293
3,2015-07-12,New South Wales,Wind speed [m/s]_mean,6.923776,6.841226
4,2005-09-10,South Australia,Wind speed [m/s]_mean,7.290148,7.300907


In [ ]:
display(change_log_df.head())
display(change_log_df.tail())
print(change_log_df["Date"].min())
print(change_log_df["Date"].max(), "\n")
print(change_log_df["Variable"].value_counts(), "\n")
# Filter to just the outlier flag columns for Precipitation and Wind speed
outlier_flag_cols = [col for col in true_isolated_train.columns 
                     if col.endswith("_is_outlier") and 
                        ("Precipitation" in col or "Wind speed" in col)]
# Count how many True values there are across all those columns (i.e. total outlier "cells")
num_outlying_variables = true_isolated_train[outlier_flag_cols].sum().sum()
print(f"Number of outlying variable instances (Precipitation & Wind speed): {num_outlying_variables}")
print(f"Total treated values: {len(change_log_df)}")

,Date,State,Variable,Original,Clipped
0,2006-04-05,Australian Capital Territory,Wind speed [m/s]_mean,6.028696,6.071348
1,2012-09-07,Australian Capital Territory,Wind speed [m/s]_mean,6.992190,6.849015
2,2013-10-10,New South Wales,Wind speed [m/s]_mean,6.046298,6.077293
3,2015-07-12,New South Wales,Wind speed [m/s]_mean,6.923776,6.841226
4,2005-09-10,South Australia,Wind speed [m/s]_mean,7.290148,7.300907


,Date,State,Variable,Original,Clipped
109,2010-10-06,Victoria,Precipitation [mm/day]_std,0.629890,0.640107
110,2011-02-18,Victoria,Precipitation [mm/day]_std,1.172752,1.115876
111,2011-11-09,Victoria,Precipitation [mm/day]_std,1.447749,1.115876
112,2005-06-17,Western Australia,Precipitation [mm/day]_std,0.358528,0.358705
113,2007-01-03,Western Australia,Precipitation [mm/day]_std,1.011720,0.784072


2005-02-09 00:00:00
2016-12-05 00:00:00 

Variable
Precipitation [mm/day]_max    26
Precipitation [mm/day]_std    26
Precipitation [mm/day]_sum    22
Precipitation [mm/day]_min    12
Wind speed [m/s]_std          12
Wind speed [m/s]_min           8
Wind speed [m/s]_mean          6
Wind speed [m/s]_max           2
Name: count, dtype: int64 

Number of outlying variable instances (Precipitation & Wind speed): 984
Total treated values: 114


#### **Save Data**

In [88]:
exp_csv_path = os.path.join(target_dir, "02_weather_metrics.csv")
era_treated.to_csv(exp_csv_path, index = True)
display(era_treated.head(5))

,State,Date,Temperature [°C]_mean,Relative humidity [%]_mean,Wind speed [m/s]_mean,Precipitation [mm/day]_sum,Solar radiation [Jm2/day]_sum,Temperature [°C]_min,Relative humidity [%]_min,Wind speed [m/s]_min,...,Temperature [°C]_max,Relative humidity [%]_max,Wind speed [m/s]_max,Precipitation [mm/day]_max,Solar radiation [Jm2/day]_max,Temperature [°C]_std,Relative humidity [%]_std,Wind speed [m/s]_std,Precipitation [mm/day]_std,Solar radiation [Jm2/day]_std
0,New South Wales,2005-01-01,27.437794,36.277184,3.351300,0.043266,26852402.0,20.457810,19.242327,1.310347,...,34.291195,54.994526,5.426612,0.025815,3545229.8,4.964612,12.250730,1.222393,0.005683,1308933.00
1,New South Wales,2005-01-02,26.196392,38.767070,3.083268,0.173695,26924144.0,19.180061,21.069553,1.240741,...,33.001564,58.154682,5.300327,0.073848,3603005.0,4.940912,12.738175,1.195053,0.016719,1295075.80
2,New South Wales,2005-01-03,27.574987,48.768177,3.936694,2.834388,25037406.0,21.237320,22.289694,1.459488,...,34.734430,80.639420,6.776006,1.024026,3525281.2,4.847450,21.154590,1.396859,0.256273,1296644.20
3,New South Wales,2005-01-04,23.166720,57.276043,4.844326,2.945442,21772314.0,17.757936,36.951770,2.998784,...,28.794777,76.287470,7.010552,1.010933,2857264.2,3.774858,12.719058,1.168850,0.255513,1018321.06
4,New South Wales,2005-01-05,22.526860,47.163494,4.013110,0.153918,27905030.0,16.342798,30.575161,2.326827,...,28.103756,67.090096,5.754541,0.044550,3458467.5,4.178695,11.873404,1.089349,0.011417,1301825.90


### **DATASET 3**: Historical NDVI

`Australia_p1-MOD13Q1-061-Statistics.csv`

`Australia_p2-MOD13Q1-061-Statistics.csv`

In [ ]:
csv_path = os.path.join(target_dir, "MOD13Q1_v061", "Australia_p1-MOD13Q1-061-Statistics.csv")
df1 = pd.read_csv(csv_path)
csv_path = os.path.join(target_dir, "MOD13Q1_v061", "Australia_p2-MOD13Q1-061-Statistics.csv")
df2 = pd.read_csv(csv_path)

#### **Filter**

In [ ]:
### FILTER ###

df         = pd.concat([df1, df2], ignore_index=True)
df["Date"] = pd.to_datetime(df["Date"])
# Map aid codes to state names
aid_map = {
    "aid0001": "New South Wales",
    "aid0002": "Victoria",
    "aid0003": "Queensland",
    "aid0004": "South Australia",
    "aid0005": "Western Australia",
    "aid0006": "Tasmania",
    "aid0007": "Northern Territory",
    "aid0008": "Australian Capital Territory"
}
df["State"] = df["aid"].map(aid_map)
rename_map  = {
    "Minimum": "NDVI_min",
    "Maximum": "NDVI_max",
    "Mean": "NDVI_mean",
    "Standard Deviation": "NDVI_std",
    "Variance": "NDVI_var"
}
df = df[["Date", "State"] + list(rename_map.keys())].rename(columns=rename_map)

# Group to monthly resolution
df["year_month"] = df["Date"].dt.to_period("M")
monthly_df = df.groupby(["State", "year_month"]).mean(numeric_only=True).reset_index()
monthly_df["Date"] = monthly_df["year_month"].dt.to_timestamp()
monthly_df.drop(columns="year_month", inplace=True)

monthly_df = monthly_df[monthly_df["Date"] != pd.Timestamp("2004-12-01")]
monthly_df = monthly_df[monthly_df["State"] != "Northern Territory"]
display(monthly_df.head())
print(monthly_df["State"].value_counts())
print(monthly_df["Date"].min())
print(monthly_df["Date"].max())

,State,NDVI_min,NDVI_max,NDVI_mean,NDVI_std,NDVI_var,Date
1,Australian Capital Territory,-0.19790,0.88345,0.509445,0.119545,0.014299,2005-01-01
2,Australian Capital Territory,-0.15275,0.86075,0.530368,0.121759,0.014825,2005-02-01
3,Australian Capital Territory,-0.16900,0.93230,0.532693,0.134944,0.018210,2005-03-01
4,Australian Capital Territory,-0.19140,0.93330,0.534206,0.142291,0.020284,2005-04-01
5,Australian Capital Territory,-0.19115,0.89785,0.511176,0.155487,0.024182,2005-05-01


State
Australian Capital Territory    192
New South Wales                 192
Queensland                      192
South Australia                 192
Tasmania                        192
Victoria                        192
Western Australia               192
Name: count, dtype: int64
2005-01-01 00:00:00
2020-12-01 00:00:00


#### **Outlier Analysis**

In [ ]:
### OUTLIER ANALYSIS ###

# Z-score & outlier detection
outlier_flags = []
for state, group in monthly_df.groupby("State"):
    state_outliers = group.copy()
    for col in [c for c in state_outliers.columns if c.startswith("NDVI_")]:
        if state_outliers[col].nunique() > 1:
            state_outliers[col + "_zscore"] = zscore(state_outliers[col], nan_policy='omit')
            state_outliers[col + "_is_outlier"] = state_outliers[col + "_zscore"].abs() > 3
    outlier_flags.append(state_outliers)

ndvi_outliers_df = pd.concat(outlier_flags, ignore_index=True)

# Outlier summary
outlier_cols = [col for col in ndvi_outliers_df.columns if col.endswith("_is_outlier")]
total_outliers = ndvi_outliers_df[outlier_cols].sum().sort_values(ascending=False)
print("Outlier counts per NDVI variable:")
print(total_outliers)

# Get list of outlier columns
outlier_cols = [col for col in ndvi_outliers_df.columns if col.endswith("_is_outlier")]

# Extract base variable names from column names
summary = []

for state, group in ndvi_outliers_df.groupby("State"):
    counts = {"State": state}
    for var in ["min", "max", "mean", "std", "var"]:
        col_name = f"NDVI_{var}_is_outlier"
        counts[f"outlier_{var}"] = group[col_name].sum() if col_name in group.columns else 0
    summary.append(counts)

state_outlier_summary = pd.DataFrame(summary)
state_outlier_summary = state_outlier_summary.set_index("State").astype(int)
display(state_outlier_summary)

Outlier counts per NDVI variable:
NDVI_max_is_outlier     25
NDVI_min_is_outlier      7
NDVI_std_is_outlier      3
NDVI_var_is_outlier      3
NDVI_mean_is_outlier     1
dtype: int64


,outlier_min,outlier_max,outlier_mean,outlier_std,outlier_var
State,,,,,
Australian Capital Territory,6,3,1,0,0
New South Wales,0,2,0,2,2
Queensland,0,2,0,0,0
South Australia,1,3,0,0,0
Tasmania,0,3,0,1,1
Victoria,0,7,0,0,0
Western Australia,0,5,0,0,0


In [ ]:
outliers_ndvi = ndvi_outliers_df.copy()
outliers_ndvi["is_outlier"] = outliers_ndvi[[f"{col}_is_outlier" for col in ["NDVI_min", "NDVI_max", "NDVI_mean", "NDVI_std", "NDVI_var"]]].any(axis=1)
outliers_ndvi = outliers_ndvi[outliers_ndvi["is_outlier"]].sort_values(["State", "Date"]).reset_index(drop=True)

# Detect streaks/clusters per state
def label_temporal_clusters(df, max_gap_months=1, min_cluster_len=2):
    df["cluster_id"] = np.nan
    cluster_id = 0
    for state, group in df.groupby("State"):
        group = group.sort_values("Date").reset_index()
        streak = []
        for i in range(len(group)):
            if i == 0 or (group.loc[i, "Date"] - group.loc[i - 1, "Date"]).days <= max_gap_months * 31:
                streak.append(i)
            else:
                if len(streak) >= min_cluster_len:
                    cluster_id += 1
                    df.loc[group.loc[streak, "index"], "cluster_id"] = cluster_id
                streak = [i]
        if len(streak) >= min_cluster_len:
            cluster_id += 1
            df.loc[group.loc[streak, "index"], "cluster_id"] = cluster_id
    df["is_temporally_clustered"] = ~df["cluster_id"].isna()
    return df

outliers_ndvi = label_temporal_clusters(outliers_ndvi)

csv_path = os.path.join(target_dir, "01_historical_wildfires.csv")
fires    = pd.read_csv(csv_path)
fires["ignition_date"] = pd.to_datetime(fires["ignition_date"])

def fire_proximity_info(ndvi_df, fires_df, max_days_window=30):
    is_near    = []
    relation   = []
    day_deltas = []
    fires_df["ignition_date"] = pd.to_datetime(fires_df["ignition_date"])
    for idx, row in ndvi_df.iterrows():
        state = row["State"]
        ndvi_date = row["Date"]
        # Filter fires for the same state
        state_fires = fires_df[fires_df["state"] == state]
        # Compute time differences (positive = fire after NDVI)
        time_deltas = (state_fires["ignition_date"] - ndvi_date).dt.days
        if time_deltas.empty:
            is_near.append(False)
            relation.append(None)
            day_deltas.append(None)
            continue
        nearest_delta = time_deltas.abs().min()

        if nearest_delta <= max_days_window:
            nearest = time_deltas[time_deltas.abs() == nearest_delta].iloc[0]
            is_near.append(True)
            day_deltas.append(nearest)
            if nearest < 0:
                relation.append("before_fire")
            elif nearest > 0:
                relation.append("after_fire")
            else:
                relation.append("same_day")
        else:
            is_near.append(False)
            relation.append(None)
            day_deltas.append(None)

    return pd.DataFrame({
        "is_near_fire": is_near,
        "fire_time_relation": relation,
        "days_to_nearest_fire": day_deltas
    })

proximity_df = fire_proximity_info(outliers_ndvi, fires, max_days_window=30)
outliers_ndvi = pd.concat([outliers_ndvi.reset_index(drop=True), proximity_df], axis=1)

print(outliers_ndvi["fire_time_relation"].value_counts(dropna=False))
print(outliers_ndvi[["Date", "State", "fire_time_relation", "days_to_nearest_fire"]].head())

print("Clustered NDVI outliers:", outliers_ndvi["is_temporally_clustered"].sum())
print("NDVI outliers near fire:", outliers_ndvi["is_near_fire"].sum())
display(outliers_ndvi[outliers_ndvi["is_temporally_clustered"] == True])
display(outliers_ndvi[outliers_ndvi["is_near_fire"] == True])

fire_time_relation
after_fire     12
before_fire     9
None            9
same_day        4
Name: count, dtype: int64
        Date                         State fire_time_relation  \
0 2006-12-01  Australian Capital Territory         after_fire   
1 2007-01-01  Australian Capital Territory        before_fire   
2 2014-06-01  Australian Capital Territory               None   
3 2018-07-01  Australian Capital Territory               None   
4 2019-02-01  Australian Capital Territory        before_fire   

   days_to_nearest_fire  
0                  23.0  
1                  -8.0  
2                   NaN  
3                   NaN  
4                  -3.0  
Clustered NDVI outliers: 8
NDVI outliers near fire: 25


,State,NDVI_min,NDVI_max,NDVI_mean,NDVI_std,NDVI_var,Date,NDVI_min_zscore,NDVI_min_is_outlier,NDVI_max_zscore,...,NDVI_std_zscore,NDVI_std_is_outlier,NDVI_var_zscore,NDVI_var_is_outlier,is_outlier,cluster_id,is_temporally_clustered,is_near_fire,fire_time_relation,days_to_nearest_fire
0,Australian Capital Territory,-0.12865,0.77025,0.426850,0.131939,0.017437,2006-12-01,2.980598,False,-3.461372,...,0.228530,False,0.149736,False,True,1.0,True,True,after_fire,23.0
1,Australian Capital Territory,-0.18175,0.77455,0.422066,0.122978,0.015124,2007-01-01,-0.057877,False,-3.371863,...,-0.228319,False,-0.302871,False,True,1.0,True,True,before_fire,-8.0
5,Australian Capital Territory,-0.13945,0.78640,0.431854,0.134723,0.018158,2019-12-01,2.362603,False,-3.125195,...,0.370412,False,0.290725,False,True,2.0,True,True,before_fire,-16.0
6,Australian Capital Territory,-0.11825,0.87055,0.427718,0.131485,0.017297,2020-01-01,3.575705,True,-1.373539,...,0.205338,False,0.122289,False,True,2.0,True,True,after_fire,6.0
7,Australian Capital Territory,-0.12640,0.92780,0.603245,0.119960,0.014390,2020-11-01,3.109347,True,-0.181830,...,-0.382210,False,-0.446522,False,True,3.0,True,False,None,NaN
8,Australian Capital Territory,-0.08875,0.96225,0.580537,0.113792,0.012956,2020-12-01,5.263747,True,0.535277,...,-0.696647,False,-0.727249,False,True,3.0,True,False,None,NaN
11,New South Wales,-0.20000,0.99905,0.272580,0.147444,0.021749,2019-12-01,-0.992278,False,-0.114904,...,-4.066778,True,-3.635240,True,True,4.0,True,True,before_fire,-1.0
12,New South Wales,-0.20000,0.99910,0.278225,0.150839,0.022904,2020-01-01,-0.992278,False,0.007660,...,-3.810914,True,-3.414291,True,True,4.0,True,True,same_day,0.0


,State,NDVI_min,NDVI_max,NDVI_mean,NDVI_std,NDVI_var,Date,NDVI_min_zscore,NDVI_min_is_outlier,NDVI_max_zscore,...,NDVI_std_zscore,NDVI_std_is_outlier,NDVI_var_zscore,NDVI_var_is_outlier,is_outlier,cluster_id,is_temporally_clustered,is_near_fire,fire_time_relation,days_to_nearest_fire
0,Australian Capital Territory,-0.12865,0.77025,0.426850,0.131939,0.017437,2006-12-01,2.980598,False,-3.461372,...,0.228530,False,0.149736,False,True,1.0,True,True,after_fire,23.0
1,Australian Capital Territory,-0.18175,0.77455,0.422066,0.122978,0.015124,2007-01-01,-0.057877,False,-3.371863,...,-0.228319,False,-0.302871,False,True,1.0,True,True,before_fire,-8.0
4,Australian Capital Territory,-0.12445,0.91735,0.560795,0.132457,0.017574,2019-02-01,3.220930,True,-0.399356,...,0.254892,False,0.176468,False,True,NaN,False,True,before_fire,-3.0
5,Australian Capital Territory,-0.13945,0.78640,0.431854,0.134723,0.018158,2019-12-01,2.362603,False,-3.125195,...,0.370412,False,0.290725,False,True,2.0,True,True,before_fire,-16.0
6,Australian Capital Territory,-0.11825,0.87055,0.427718,0.131485,0.017297,2020-01-01,3.575705,True,-1.373539,...,0.205338,False,0.122289,False,True,2.0,True,True,after_fire,6.0
9,New South Wales,-0.20000,0.99515,0.427690,0.218495,0.047750,2008-09-01,-0.992278,False,-9.674919,...,1.286849,False,1.337281,False,True,NaN,False,True,after_fire,22.0
10,New South Wales,-0.20000,0.99720,0.401142,0.202250,0.040941,2009-09-01,-0.992278,False,-4.649783,...,0.062789,False,0.035151,False,True,NaN,False,True,after_fire,25.0
11,New South Wales,-0.20000,0.99905,0.272580,0.147444,0.021749,2019-12-01,-0.992278,False,-0.114904,...,-4.066778,True,-3.635240,True,True,4.0,True,True,before_fire,-1.0
12,New South Wales,-0.20000,0.99910,0.278225,0.150839,0.022904,2020-01-01,-0.992278,False,0.007660,...,-3.810914,True,-3.414291,True,True,4.0,True,True,same_day,0.0
13,Queensland,-0.20000,0.99835,0.338714,0.171442,0.029393,2008-07-01,-0.992278,False,-7.542520,...,0.247529,False,0.194493,False,True,NaN,False,True,before_fire,-1.0


In [ ]:
### OUTLIER TREATMENT ###

ndvi_treated = monthly_df.copy()

clip_low  = 0.05
clip_high = 0.95
treatment_variables = ["NDVI_min", "NDVI_max", "NDVI_std", "NDVI_var", "NDVI_mean"]
change_log = []

# Define which ones are true isolated (already filtered earlier)
true_isolated_outliers = outliers_ndvi[
    (outliers_ndvi["is_outlier"]) &
    (~outliers_ndvi["is_temporally_clustered"]) &
    (~outliers_ndvi["is_near_fire"])
].copy()
# Filter to training years only (2005–2016)
true_isolated_outliers = true_isolated_outliers[
    (true_isolated_outliers["Date"].dt.year >= 2005) &
    (true_isolated_outliers["Date"].dt.year <= 2016)
]
# Loop by state first, then variables
for state, group in true_isolated_outliers.groupby("State"):
    print(f"\nProcessing state: {state}")
    for var in treatment_variables:
        outlier_col = f"{var}_is_outlier"
        if outlier_col not in group.columns:
            continue
        # Get valid outliers for this variable in this state
        var_outliers = group[
            group[outlier_col].fillna(False) &
            group[var].notna()
        ]
        if var_outliers.empty or var_outliers.shape[0] < 3:
            print(f"  Skipping {var} — Not enough data.")
            continue
        # Compute state-specific clipping bounds for this variable
        lower = var_outliers[var].quantile(clip_low)
        upper = var_outliers[var].quantile(clip_high)

        print(f"  {var}: Clipping to [{lower:.4f}, {upper:.4f}] for {len(var_outliers)} rows")

        for idx, row in var_outliers.iterrows():
            date    = row["Date"]
            value   = row[var]
            clipped = np.clip(value, lower, upper)
            if clipped != value:
                ndvi_treated.loc[
                    (ndvi_treated["Date"] == date) & (ndvi_treated["State"] == state),
                    var
                ] = clipped

                change_log.append({
                    "State": state,
                    "Date": date,
                    "Variable": var,
                    "Original": value,
                    "Clipped": clipped
                })

change_log_df = pd.DataFrame(change_log)

print(f"\nTotal values treated: {len(change_log_df)}")
display(change_log_df.head())

exp_csv_path = os.path.join(target_dir, "03b_change_log.csv")
change_log_df.to_csv(exp_csv_path, index=True)


Processing state: Australian Capital Territory
  Skipping NDVI_min — Not enough data.
  Skipping NDVI_max — Not enough data.
  Skipping NDVI_std — Not enough data.
  Skipping NDVI_var — Not enough data.
  Skipping NDVI_mean — Not enough data.

Processing state: Tasmania
  Skipping NDVI_min — Not enough data.
  Skipping NDVI_max — Not enough data.
  Skipping NDVI_std — Not enough data.
  Skipping NDVI_var — Not enough data.
  Skipping NDVI_mean — Not enough data.

Processing state: Western Australia
  Skipping NDVI_min — Not enough data.
  NDVI_max: Clipping to [0.9950, 0.9967] for 3 rows
  Skipping NDVI_std — Not enough data.
  Skipping NDVI_var — Not enough data.
  Skipping NDVI_mean — Not enough data.

Total values treated: 2


,State,Date,Variable,Original,Clipped
0,Western Australia,2007-06-01,NDVI_max,0.99680,0.996750
1,Western Australia,2008-08-01,NDVI_max,0.99485,0.994995


#### **Save Data**

In [89]:
exp_csv_path = os.path.join(target_dir, "03_NDVI.csv")
ndvi_treated.to_csv(exp_csv_path, index = True)
display(ndvi_treated.head(5))

,State,NDVI_min,NDVI_max,NDVI_mean,NDVI_std,NDVI_var,Date
1,Australian Capital Territory,-0.19790,0.88345,0.509445,0.119545,0.014299,2005-01-01
2,Australian Capital Territory,-0.15275,0.86075,0.530368,0.121759,0.014825,2005-02-01
3,Australian Capital Territory,-0.16900,0.93230,0.532693,0.134944,0.018210,2005-03-01
4,Australian Capital Territory,-0.19140,0.93330,0.534206,0.142291,0.020284,2005-04-01
5,Australian Capital Territory,-0.19115,0.89785,0.511176,0.155487,0.024182,2005-05-01
